In [3]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, mnist32_cnn
from libs.constants import model_seeds
from libs.fitnessfns import ff_lenet_3hidden_ERRexpbitflips
from timeit import default_timer as timer

In [4]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [5]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)
all_images = np.expand_dims(all_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
all_images = tf.image.resize(all_images, [image_x_size, image_y_size]) 

In [6]:
model_type = "mnist32-cnn_1024_256_64"
model_seed = model_seeds[0]

In [7]:
# Get model
model_instance = model_type + "-" + str(model_seed)
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)

model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_meta_type / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [8]:
def eval_error_injected_no_shuffle(model, 
                                  error_layer, 
                                  error_profile, 
                                  ERR_PARAM,
                                  test_set,
                                  batchsize):
    
    error_profile_c0=None
    error_profile_h0=None
    error_profile_h1=None
    error_profile_h2=None
    error_profile_op=None
    if error_layer == "c0":
        error_profile_c0=error_profile
    elif error_layer == "h0":
        error_profile_h0=error_profile
    elif error_layer =="h1":
        error_profile_h1=error_profile
    elif error_layer == "h2":
        error_profile_h2=error_profile
    elif error_layer == "op":
        error_profile_op=error_profile
    elif error_layer == "all":
        error_profile_c0=error_profile
        error_profile_h0=error_profile
        error_profile_h1=error_profile
        error_profile_h2=error_profile
        error_profile_op=error_profile
        
    acc_list = []
    for _ in range(3):
        accuracy = ff_lenet_3hidden_ERRexpbitflips(model,
                                                    error_profile_c0,
                                                    error_profile_h0,
                                                    error_profile_h1,
                                                    error_profile_h2,
                                                    error_profile_op,
                                                    ERR_PARAM=ERR_PARAM,
                                                    clayer0_shuffle_order=None,
                                                    hlayer0_shuffle_order=None,
                                                    hlayer1_shuffle_order=None,
                                                    hlayer2_shuffle_order=None,
                                                    oplayer_shuffle_order=None,
                                                    test_set=test_set,
                                                    batchsize=batchsize)
        acc_list.append(accuracy)
    return np.mean(acc_list)

In [9]:
# Load model
model = tf.keras.models.load_model(model_file)
# model.summary()

# Load test set
# Testing with only the last 12_8000 images
test_set = (all_images[-12_800:], all_labels[-12_800:])

In [12]:
model.layers[5].output_shape[-1]

64

In [8]:
# Original Model
for _ in range(1):
    start = timer()
    accuracy = ff_lenet_3hidden_ERRexpbitflips(model,
                                                error_profile_c0=None,
                                                error_profile_h0=None,
                                                error_profile_h1=None,
                                                error_profile_h2=None,
                                                error_profile_op=None,
                                                ERR_PARAM=None,
                                                clayer0_shuffle_order=None,
                                                hlayer0_shuffle_order=None,
                                                hlayer1_shuffle_order=None,
                                                hlayer2_shuffle_order=None,
                                                oplayer_shuffle_order=None,
                                                test_set=test_set,
                                                batchsize=128)
    end = timer()
    print(accuracy, end - start) # Time in seconds,

tf.Tensor(0.98859375, shape=(), dtype=float64) 4.703765101032332


In [9]:
results = {}
for error_profile_tag in ['LIM_01-2188', 'LIM_01-3987', 'LIM_05-2188', 'LIM_05-3987']:
    results[error_profile_tag] = {}
    # Load error profile
    err_profile_filename = "../error_profiles/"+ error_profile_tag + ".npy"
    error_profile = np.load(err_profile_filename)

    ERR_PARAM_list = [0,1,-1]
    error_layer_list = ["c0", "h0", "h1", "h2", "op", "all"]
    EVAL_BATCHSIZE = 128
    for ERR_PARAM in ERR_PARAM_list:
        results[error_profile_tag][ERR_PARAM]={}
        for error_layer in error_layer_list:
            eval_acc = eval_error_injected_no_shuffle(model, 
                                                      error_layer, 
                                                      error_profile, 
                                                      ERR_PARAM,
                                                      test_set,
                                                      EVAL_BATCHSIZE)
            results[error_profile_tag][ERR_PARAM][error_layer] = np.around(eval_acc,decimals=4)
            np.save('fault_injection_analysis_results.npy', results) 
            print(ERR_PARAM, '\t' ,error_layer, '\t' , np.around(eval_acc,decimals=4))
        print('*'*50)    
    print('*'*50)
    print('*'*50)
    print()

0 	 c0 	 0.9886
0 	 h0 	 0.9886
0 	 h1 	 0.9886
0 	 h2 	 0.9886
0 	 op 	 0.9886
0 	 all 	 0.9883
**************************************************
1 	 c0 	 0.4408
1 	 h0 	 0.9868
1 	 h1 	 0.9883
1 	 h2 	 0.9194
1 	 op 	 0.9715
1 	 all 	 0.3917
**************************************************
-1 	 c0 	 0.4363
-1 	 h0 	 0.9865
-1 	 h1 	 0.9881
-1 	 h2 	 0.9183
-1 	 op 	 0.9728
-1 	 all 	 0.383
**************************************************
**************************************************
**************************************************

0 	 c0 	 0.9887
0 	 h0 	 0.9887
0 	 h1 	 0.9884
0 	 h2 	 0.9885
0 	 op 	 0.9886
0 	 all 	 0.9882
**************************************************
1 	 c0 	 0.3361
1 	 h0 	 0.9868
1 	 h1 	 0.9887
1 	 h2 	 0.9199
1 	 op 	 0.9707
1 	 all 	 0.2875
**************************************************
-1 	 c0 	 0.325
-1 	 h0 	 0.9867
-1 	 h1 	 0.9884
-1 	 h2 	 0.9197
-1 	 op 	 0.9713
-1 	 all 	 0.2871
**************************************************

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 fc_0 (Dense)                (None, 1024)              6423552   
                                                                 
 fc_1 (Dense)                (None, 256)               262400    
                                                                 
 fc_2 (Dense)                (None, 64)                16448     
                                                        

In [10]:
# # Error Injected Model [No shuffling]
# for _ in range(3):
#     start = timer()
#     accuracy = ff_lenet_3hidden_ERRexpbitflips(model,
#                                                 error_profile_c0=error_profile,
#                                                 error_profile_h0=error_profile,
#                                                 error_profile_h1=error_profile,
#                                                 error_profile_h2=error_profile,
#                                                 error_profile_op=error_profile,
#                                                 ERR_PARAM=-1,
#                                                 clayer0_shuffle_order=None,
#                                                 hlayer0_shuffle_order=None,
#                                                 hlayer1_shuffle_order=None,
#                                                 hlayer2_shuffle_order=None,
#                                                 oplayer_shuffle_order=None,
#                                                 test_set=test_set,
#                                                 batchsize=128)
#     end = timer()
#     print(accuracy, end - start) # Time in seconds,

In [11]:
# # Error Injected Model [Random shuffling]
# # Load shuffle order
# clayer0_shuffle_order = np.arange(32)
# hlayer0_shuffle_order = np.arange(1024)
# hlayer1_shuffle_order = np.arange(256)
# hlayer2_shuffle_order = np.arange(64)
# oplayer_shuffle_order = np.arange(10)

# np.random.shuffle(clayer0_shuffle_order)
# np.random.shuffle(hlayer0_shuffle_order)
# np.random.shuffle(hlayer1_shuffle_order)
# np.random.shuffle(hlayer2_shuffle_order)
# np.random.shuffle(oplayer_shuffle_order)

# for _ in range(3):
#     start = timer()
#     accuracy = ff_lenet_3hidden_ERRexpbitflips(model,
#                                                 error_profile_c0=error_profile,
#                                                 error_profile_h0=error_profile,
#                                                 error_profile_h1=error_profile,
#                                                 error_profile_h2=error_profile,
#                                                 error_profile_op=error_profile,
#                                                 ERR_PARAM=-1,
#                                                 clayer0_shuffle_order=clayer0_shuffle_order,
#                                                 hlayer0_shuffle_order=hlayer0_shuffle_order,
#                                                 hlayer1_shuffle_order=hlayer1_shuffle_order,
#                                                 hlayer2_shuffle_order=hlayer2_shuffle_order,
#                                                 oplayer_shuffle_order=oplayer_shuffle_order,
#                                                 test_set=test_set,
#                                                 batchsize=128)
#     end = timer()
#     print(accuracy, end - start) # Time in seconds,